In [1]:
import sqlite3
import pandas as pd

Create database

In [2]:
class WeatherDatabase:
    url: str

    def __init__(self, url: str, *args):
        self.url = url
        con = sqlite3.connect(url)
        cur = con.cursor()
        for table in args:
            con.execute(
                f"""
                CREATE DATABASE IF NOT EXISTS Expences
                """)
        cur.close()
        con.commit()
        con.close()

    def call_db(self, query, *args):
        con = sqlite3.connect(self.url)
        cur = con.cursor()
        res = cur.execute(query, args)
        data = res.fetchall()
        cur.close()
        con.commit()
        con.close()
        return data
    

In [3]:
db = WeatherDatabase('./WeatherGBG.db')

Create table for average daily temperatures, full dataset

In [4]:
db.call_db("""
    CREATE TABLE IF NOT EXISTS avg_daily_temp_data
(   id INTEGER PRIMARY KEY AUTOINCREMENT,
    Date DATE,
    Quality TEXT,
    Avg temp FLOAT,
    Year DATE
)
""")

[]

In [5]:
insert_into_avg_daily_temp_table = """
INSERT INTO avg_daily_temp_data
    (
    Date, 
    Quality,
    Avg,
    Year
    )
VALUES
    (
    ?, ?, ?, ?
    )

"""

In [6]:
df = pd.read_csv('../data/Olof_Save_GBG_avg_daily_temp_data.csv')
for i in df.itertuples(index=False):
        db.call_db(insert_into_avg_daily_temp_table, str(i.Datum), str(i.Kvalitet), float(i.Snittemperatur), str(i.Year))

Create table for average montly temperatures, full dataset

In [17]:
db.call_db("""
    CREATE TABLE IF NOT EXISTS avg_monthly_temp_data
(   id INTEGER PRIMARY KEY AUTOINCREMENT,
    Year INT,
    Month INT, 
    Monthly_avg FLOAT
)
""")

[]

In [3]:
# db.call_db("""DROP TABLE avg_monthly_temp_data""" )

In [18]:
insert_into_avg_monthly_temp_data = """
INSERT INTO avg_monthly_temp_data
    (
    Year, 
    Month, 
    Monthly_avg
    )
VALUES
    (
    ?, ?, ?
    )

"""

In [19]:
df = pd.read_csv('../data/Olof_Save_GBG_avg_monthly_temp_data.csv')
for i in df.itertuples(index=False):
        db.call_db(insert_into_avg_monthly_temp_data, int(i.Year), int(i.Month), float(i.Monthly_avg_temp))

Create table with raw temperature data, GBG

In [25]:
df_raw_gbg = pd.read_csv('../data/smhi_temp_data_gbg.csv', sep=';', usecols=[0, 1, 2, 3], header=11)
df_raw_gbg = df_raw_gbg.rename(columns=({'Tid (UTC)': 'Tid_UTC'}))
df_raw_gbg
# header countr only populated rows

,Datum,Tid_UTC,Lufttemperatur,Kvalitet
0,1961-01-01,06:00:00,0.8,G
1,1961-01-01,12:00:00,1.0,G
2,1961-01-01,18:00:00,1.4,G
3,1961-01-02,06:00:00,1.8,G
4,1961-01-02,12:00:00,2.2,G
...,...,...,...,...
308145,2023-07-01,02:00:00,16.1,G
308146,2023-07-01,03:00:00,16.1,G
308147,2023-07-01,04:00:00,16.2,G
308148,2023-07-01,05:00:00,16.3,G


In [15]:
db.call_db("""
    CREATE TABLE IF NOT EXISTS temp_GBG_raw
(   id INTEGER PRIMARY KEY AUTOINCREMENT,
    Datum STR,
    Tid_UTC STR, 
    Lufttemperatur FLOAT,
    Kvalitet STR
)
""")

[]

In [27]:
insert_into_temp_GBG_raw = """
INSERT INTO temp_GBG_raw
    (
    Datum, 
    Tid_UTC, 
    Lufttemperatur,
    Kvalitet
    )
VALUES
    (
    ?, ?, ?, ?
    )

"""

In [28]:
for i in df_raw_gbg.itertuples(index=False):
    db.call_db(insert_into_temp_GBG_raw, str(i.Datum), str(i.Tid_UTC), float(i.Lufttemperatur), str(i.Kvalitet))

In [ ]:
add_colum = """ALTER TABLE Temp_GBG_raw
                ADD COLUMN Datum_date DATE"""
db.call_db(add_colum)

In [ ]:
add_dates_to_column = """UPDATE Temp_GBG_raw
                            SET Datum_date = DATE(Datum)"""
db.call_db(add_dates_to_column)

Create table with raw temperature data, Säve

In [12]:
df_raw_temp_data_save = pd.read_csv('../data/smhi_temp_data_save.csv', usecols=[0, 1, 2, 3], sep=';', header=6)
df_raw_temp_data_save = df_raw_temp_data_save.rename(columns=({'Tid (UTC)': 'Tid_UTC'}))
df_raw_temp_data_save

,Datum,Tid_UTC,Lufttemperatur,Kvalitet
0,1944-01-01,07:00:00,1.6,G
1,1944-01-01,13:00:00,3.8,G
2,1944-01-01,18:00:00,4.2,G
3,1944-01-02,07:00:00,0.6,G
4,1944-01-02,13:00:00,3.8,G
...,...,...,...,...
435387,2006-12-05,09:00:00,9.4,G
435388,2006-12-05,10:00:00,10.1,Y
435389,2006-12-05,11:00:00,10.8,Y
435390,2006-12-05,12:00:00,10.9,G


In [ ]:
db.call_db("""
    CREATE TABLE IF NOT EXISTS temp_Save_raw
(   id INTEGER PRIMARY KEY AUTOINCREMENT,
    Datum STR,
    Tid_UTC STR, 
    Lufttemperatur FLOAT,
    Kvalitet STR
)
""")

In [ ]:
insert_into_temp_Save_raw = """
INSERT INTO temp_Save_raw
    (
    Datum, 
    Tid_UTC, 
    Lufttemperatur,
    Kvalitet
    )
VALUES
    (
    ?, ?, ?, ?
    )

"""

In [ ]:
for i in df_raw_temp_data_save.itertuples(index=False):
    db.call_db(insert_into_temp_Save_raw, str(i.Datum), str(i.Tid_UTC), float(i.Lufttemperatur), str(i.Kvalitet))

In [ ]:
add_colum = """ALTER TABLE Temp_Save_raw
                ADD COLUMN Datum_date DATE"""
db.call_db(add_colum)
add_dates_to_column = """UPDATE Temp_Save_raw
                            SET Datum_date = DATE(Datum)"""
db.call_db(add_dates_to_column)

Create table for raw precipitation data, GBG

In [14]:
df_precipitation_raw_gbg = pd.read_csv('../data/smhi_percipitation_data.csv', sep=';', usecols=[0, 1, 2, 3, 4], header=11)
df_precipitation_raw_gbg = df_precipitation_raw_gbg.rename(columns=({'Från Datum Tid (UTC)': 'Från_datum_tid_UTC', 'Till Datum Tid (UTC)': 'Till_datum_tid_UTC', 'Representativt dygn': 'Representativt_dygn'}))
df_precipitation_raw_gbg

,Från_datum_tid_UTC,Till_datum_tid_UTC,Representativt_dygn,Nederbördsmängd,Kvalitet
0,1961-01-01 06:00:01,1961-01-02 06:00:00,1961-01-01,0.3,Y
1,1961-01-02 06:00:01,1961-01-03 06:00:00,1961-01-02,0.5,Y
2,1961-01-03 06:00:01,1961-01-04 06:00:00,1961-01-03,8.4,Y
3,1961-01-04 06:00:01,1961-01-05 06:00:00,1961-01-04,0.0,Y
4,1961-01-05 06:00:01,1961-01-06 06:00:00,1961-01-05,2.0,Y
...,...,...,...,...,...
15487,2023-06-26 06:00:01,2023-06-27 06:00:00,2023-06-26,18.5,G
15488,2023-06-27 06:00:01,2023-06-28 06:00:00,2023-06-27,10.6,G
15489,2023-06-28 06:00:01,2023-06-29 06:00:00,2023-06-28,0.0,G
15490,2023-06-29 06:00:01,2023-06-30 06:00:00,2023-06-29,5.0,G


In [20]:
db.call_db("""
    CREATE TABLE IF NOT EXISTS precipitation_data_GBG_raw
(   id INTEGER PRIMARY KEY AUTOINCREMENT,
    Tid_from_UTC STR,
    Tid_tom_UTC STR, 
    Representativt_dygn STR,
    Nederbördsmängd FLOAT,
    Kvalitet STR
)
""")

[]

In [21]:
insert_into_precipitation_GBG_raw = """
INSERT INTO precipitation_data_GBG_raw
    (
    Tid_from_UTC,
    Tid_tom_UTC, 
    Representativt_dygn,
    Nederbördsmängd,
    Kvalitet
    )
VALUES
    (
    ?, ?, ?, ?, ?
    )

"""

In [22]:
for i in df_precipitation_raw_gbg.itertuples(index=False):
    db.call_db(insert_into_precipitation_GBG_raw, str(i.Från_datum_tid_UTC), str(i.Till_datum_tid_UTC), str(i.Representativt_dygn), float(i.Nederbördsmängd), str(i.Kvalitet))

In [ ]:
add_colum_gbg = """ALTER TABLE precipitation_data_GBG_raw
                ADD COLUMN Datum_date DATE"""
db.call_db(add_colum_gbg)
add_dates_to_column_gbg = """UPDATE precipitation_data_GBG_raw
                            SET Datum_date = DATE(Representativt_dygn)"""
db.call_db(add_dates_to_column_gbg)

Create table for raw precipitation data, Säve


In [24]:
df_precipitation_raw_save = pd.read_csv('../data/smhi_precipitation_save.csv', sep=';', usecols=[0, 1, 2, 3, 4], header=6)
df_precipitation_raw_save = df_precipitation_raw_save.rename(columns=({'Från Datum Tid (UTC)': 'Från_datum_tid_UTC', 'Till Datum Tid (UTC)': 'Till_datum_tid_UTC', 'Representativt dygn': 'Representativt_dygn'}))
df_precipitation_raw_save

,Från_datum_tid_UTC,Till_datum_tid_UTC,Representativt_dygn,Nederbördsmängd,Kvalitet
0,1944-01-01 07:00:01,1944-01-02 07:00:00,1944-01-01,0.2,G
1,1944-01-02 07:00:01,1944-01-03 07:00:00,1944-01-02,0.4,G
2,1944-01-03 07:00:01,1944-01-04 07:00:00,1944-01-03,0.0,G
3,1944-01-04 07:00:01,1944-01-05 07:00:00,1944-01-04,0.0,G
4,1944-01-05 07:00:01,1944-01-06 07:00:00,1944-01-05,2.1,G
...,...,...,...,...,...
21514,2002-11-26 06:00:01,2002-11-27 06:00:00,2002-11-26,4.1,G
21515,2002-11-27 06:00:01,2002-11-28 06:00:00,2002-11-27,0.0,G
21516,2002-11-28 06:00:01,2002-11-29 06:00:00,2002-11-28,3.9,G
21517,2002-11-29 06:00:01,2002-11-30 06:00:00,2002-11-29,7.1,G


In [25]:
db.call_db("""
    CREATE TABLE IF NOT EXISTS precipitation_data_Save_raw
(   id INTEGER PRIMARY KEY AUTOINCREMENT,
    Tid_from_UTC STR,
    Tid_tom_UTC STR, 
    Representativt_dygn STR,
    Nederbördsmängd FLOAT,
    Kvalitet STR
)
""")

[]

In [27]:
insert_into_precipitation_Save_raw = ("""
INSERT INTO precipitation_data_Save_raw
    (
    Tid_from_UTC,
    Tid_tom_UTC, 
    Representativt_dygn,
    Nederbördsmängd,
    Kvalitet
    )
VALUES
    (
    ?, ?, ?, ?, ?
    )

""")

In [28]:
for i in df_precipitation_raw_save.itertuples(index=False):
    db.call_db(insert_into_precipitation_Save_raw, str(i.Från_datum_tid_UTC), str(i.Till_datum_tid_UTC), str(i.Representativt_dygn), float(i.Nederbördsmängd), str(i.Kvalitet))

In [ ]:
add_colum_save = """ALTER TABLE precipitation_data_save_raw
                ADD COLUMN Datum_date DATE"""
db.call_db(add_colum_save)
add_dates_to_column_save = """UPDATE precipitation_data_save_raw
                            SET Datum_date = DATE(Representativt_dygn)"""
db.call_db(add_dates_to_column_save)